In [27]:
import os
import librosa
# import pydub
from pydub import AudioSegment
import numpy as np
import copy
import random
from scipy import signal as scipy_signal
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim

import datetime

In [2]:
dataset_dir = '/home/torooc/dataHDD2/speech_commands_v0.01/'

In [3]:
testing_list_path = dataset_dir + 'testing_list.txt'
validation_list_path = dataset_dir + 'validation_list.txt'

In [4]:
with open(testing_list_path) as f:
    testing_list = f.readlines()
    testing_list = [file.strip() for file in testing_list]
    
with open(validation_list_path) as f:
    validation_list = f.readlines()
    validation_list = [file.strip() for file in validation_list]

In [5]:
command_dirs = os.listdir(dataset_dir)

command_dirs = [direc for direc in command_dirs if os.path.isdir(os.path.join(dataset_dir, direc))]

command_dirs.sort()

command_dirs.remove('_background_noise_')

In [6]:
print(command_dirs)

['bed', 'bird', 'cat', 'dog', 'down', 'eight', 'five', 'four', 'go', 'happy', 'house', 'left', 'marvin', 'nine', 'no', 'off', 'on', 'one', 'right', 'seven', 'sheila', 'six', 'stop', 'three', 'tree', 'two', 'up', 'wow', 'yes', 'zero']


In [7]:
key2label_dict = dict()
label2key_list = list()

key2label_dict[' '] = 0
label2key_list.append(' ')

for i, command in enumerate(command_dirs):
    key2label_dict[command] = i + 1
    label2key_list.append(command)

In [8]:
key2label_dict

{' ': 0,
 'bed': 1,
 'bird': 2,
 'cat': 3,
 'dog': 4,
 'down': 5,
 'eight': 6,
 'five': 7,
 'four': 8,
 'go': 9,
 'happy': 10,
 'house': 11,
 'left': 12,
 'marvin': 13,
 'nine': 14,
 'no': 15,
 'off': 16,
 'on': 17,
 'one': 18,
 'right': 19,
 'seven': 20,
 'sheila': 21,
 'six': 22,
 'stop': 23,
 'three': 24,
 'tree': 25,
 'two': 26,
 'up': 27,
 'wow': 28,
 'yes': 29,
 'zero': 30}

In [9]:
total_wav_file_list = list()

for command_dir in command_dirs:
    wav_list = os.listdir(os.path.join(dataset_dir, command_dir))
    
    wav_list_with_prefix = [os.path.join(command_dir, file_name) for file_name in wav_list]
    
    total_wav_file_list += wav_list_with_prefix

In [10]:
training_list = list(set(total_wav_file_list) - set(testing_list) - set(validation_list))
training_list.sort()


In [11]:
'''
validation_list
testing_list
training_list
'''

'\nvalidation_list\ntesting_list\ntraining_list\n'

In [12]:
audio_metadata = list()

for wav_file_name in total_wav_file_list:
    audio = AudioSegment.from_wav(dataset_dir + wav_file_name)
    fs = audio.frame_rate
    length_in_samples = audio.frame_count()
    
#   normalized_audio_array = np.asarray(audio.get_array_of_samples()) / 2 ** 15
    
#   print("Max: {}, Min: {}".format(max(normalized_audio_array), min(normalized_audio_array)))
    
    audio_metadata.append([fs, length_in_samples])

In [13]:
SR = 16000

class DatasetLoader():
    
    def __init__(self, batch_size, dataset_dir, wav_file_name_list, key2label, label2key):
        self.batch_size = batch_size
        self.dataset_dir = dataset_dir
        self.wav_file_name_list = copy.deepcopy(wav_file_name_list)
        self.sr = SR
        self.nsc_in_ms = 40
#         self.nov_in_ms = self.nsc_in_ms / 2
        self.nov_in_ms = 0
        self.nsc_in_sample = int(self.nsc_in_ms / 1000 * self.sr)
        self.nov_in_sample = int(self.nov_in_ms / 1000 * self.sr)
        
        self.key2label = key2label
        self.label2key = label2key
        
    def shuffle_dataset_order(self):
        random.shuffle(self.wav_file_name_list)
        
    def batch_generator(self):
        
        self.shuffle_dataset_order()
        
        spectrogram_list = list()
        label_list = list()
        
        for i, wav_file_name in enumerate(self.wav_file_name_list):
            
            audio = AudioSegment.from_wav(self.dataset_dir + wav_file_name)
            
            normalized_audio_array = np.asarray(audio.get_array_of_samples()) / 2 ** 15
    
            f, t, Zxx = scipy_signal.stft(normalized_audio_array, fs=self.sr, 
                                          nperseg=self.nsc_in_sample,
                                          noverlap=self.nov_in_sample)
        
            Sxx = np.abs(Zxx)
            
            normalized_spectrogram = (20 * np.log10(np.maximum(Sxx, 1e-8)) + 160) / 160
            spectrogram_list.append(normalized_spectrogram)
            
            keyword = wav_file_name.split('/')[0]
            label = self.key2label[keyword]
            label_list.append(label)
            

            if (i + 1) % self.batch_size == 0 or i + 1 == len(self.wav_file_name_list):
                
                spectrogram_time_step_list = [specgram.shape[1] for specgram in spectrogram_list]
                
                max_time_step = max(spectrogram_time_step_list)
                
                freq_size = spectrogram_list[0].shape[0]
                
                batch = np.zeros([len(spectrogram_list), freq_size, max_time_step])
                
                for j, specgram in enumerate(spectrogram_list):
                    batch[j, :specgram.shape[0], :specgram.shape[1]] = specgram
                
                spectrogram_list = list()
                
#                 print('{}:{}'.format(i, batch.shape[0]))
            
                yield batch, np.asarray(label_list)
                
                label_list = list()
                
#                 break
    
#             break

In [14]:
len(training_list)

51088

In [15]:
dataset_loader_training = DatasetLoader(256, dataset_dir, training_list, key2label_dict, label2key_list)
dataset_loader_testing = DatasetLoader(256, dataset_dir, testing_list, key2label_dict, label2key_list)
dataset_loader_validation = DatasetLoader(256, dataset_dir, validation_list, key2label_dict, label2key_list)

# dataset_loader_testing = DatasetLoader(64, dataset_dir, testing_list, key2label_dict, label2key_list)
# dataset_loader_validation = DatasetLoader(64, dataset_dir, validation_list, key2label_dict, label2key_list)

# dataset_loader_training = DatasetLoader(768, dataset_dir, training_list, key2label_dict, label2key_list)
# dataset_loader_testing = DatasetLoader(768, dataset_dir, testing_list, key2label_dict, label2key_list)
# dataset_loader_validation = DatasetLoader(768, dataset_dir, validation_list, key2label_dict, label2key_list)

In [16]:
# batch_generator = dataset_loader_training.batch_generator()

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [18]:
# class basic_model(nn.Module):
#     def __init__(self, D_in, H, num_layers, num_labels):
#         super(basic_model, self).__init__()
#         self.fc = torch.nn.Linear(D_in, H)
#         self.relu = torch.nn.ReLU()
#         self.dropout = nn.Dropout(p=0.2)

#         self.gru_layers = nn.ModuleList([nn.GRU(H, int(H / 2), bidirectional=True, batch_first=True) for i in range(num_layers)])

#         self.fc_pred = nn.Linear(H, num_labels)
#         self.log_softmax = nn.LogSoftmax(dim=2)
        
#     def forward(self, input_tensor):
#         # (B, T, F)
#         output_tensor = self.fc(input_tensor)
#         output_tensor = self.relu(output_tensor)
#         output_tensor = self.dropout(output_tensor)
#         # (B, T, H)
#         for layer in self.gru_layers:
#             output_tensor, _ = layer(output_tensor)
            
#         output_tensor = self.fc_pred(output_tensor)

#         output_tensor = self.log_softmax(output_tensor)
        
#         return output_tensor

class basic_conv_model(nn.Module):
#     def __init__(self, first_kernel_size, second_kernel_size, dropout_rate, num_labels):
    def __init__(self):
        super(basic_conv_model, self).__init__()
        self.conv2d_1 = nn.Conv2d(1, 16, (13, 2))
        self.conv2d_2 = nn.Conv2d(16, 32, (13, 2))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(4800, 31)
        
    def forward(self, input_tensor):
        input_tensor.unsqueeze_(1)
        tensor = self.conv2d_1(input_tensor)
#         print('[After 1st Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True)
        tensor = F.max_pool2d(tensor, (3, 2))
#         print('[After 1st maxpool2d]: {}'.format(tensor.shape))
        tensor = self.conv2d_2(tensor)
#         print('[After 2nd Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True)
        tensor = F.max_pool2d(tensor, (3, 2))
#         print('[After 2nd maxpool2d]: {}'.format(tensor.shape))
        tensor = self.flatten(tensor)
#         print('[After Flatten]: {}'.format(tensor.shape))
        tensor = self.fc(tensor)
#         print('[After fc]: {}'.format(tensor.shape))
        pred_tensor = F.log_softmax(tensor, dim=-1)
        
        return pred_tensor
    
class basic_crnn_model(nn.Module):
#     def __init__(self, first_kernel_size, second_kernel_size, dropout_rate, num_labels):
    def __init__(self):
        super(basic_crnn_model, self).__init__()
        self.conv2d = nn.Conv2d(1, 16, (13, 2))
        self.gru_1 = nn.GRU(16 * 103, 128, 1)
        self.gru_2 = nn.GRU(128, 128, 1)
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(128 * 12, 31)
        
    def forward(self, input_tensor):
        input_tensor.unsqueeze_(1)  # (B, 1, F, T)
        tensor = self.conv2d(input_tensor) # (B, 16, F, T)
#         print('[After 1st Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True)
        tensor = F.max_pool2d(tensor, (3, 2))
#         print('[After 1st maxpool2d]: {}'.format(tensor.shape))
        
        tensor = tensor.view(tensor.shape[0], -1, tensor.shape[3])
        # (B, 16 * F, T)
        
#         print('[After Reshape]: {}'.format(tensor.shape))
        
        tensor.transpose_(0, 2) # (T, F, B)
        tensor.transpose_(1, 2) # (T, B, F)

        tensor, _ = self.gru_1(tensor)
#         print('[After 2nd Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True) # (T, B, F)
#         tensor = F.max_pool2d(tensor, (3, 2))
#         print('[After 2nd maxpool2d]: {}'.format(tensor.shape))

        tensor, _ = self.gru_2(tensor)
#         print('[After 2nd Conv2d]: {}'.format(tensor.shape))
        tensor = F.relu(tensor)
        tensor = F.dropout(tensor, 0.1, training=True) # (T, B, F)

        tensor.transpose_(0, 1) # (B, T, F)
    
#         print('[After Reshape]: {}'.format(tensor.shape))
    
        tensor = self.flatten(tensor)
#         print('[After Flatten]: {}'.format(tensor.shape))
        tensor = self.fc(tensor)
#         print('[After fc]: {}'.format(tensor.shape))
        pred_tensor = F.log_softmax(tensor, dim=-1)
        
        return pred_tensor

    
# model = basic_model(321, 512, 3, 31).float().to(device)
# optimizer = optim.Adam(model.parameters(), lr=1e-4)
# dictation_loss = nn.CTCLoss().to(device)

model = basic_crnn_model().float().to(device)  
optimizer = optim.Adam(model.parameters(), lr=1e-4)
        

In [19]:
# batch_generator = dataset_loader_training.batch_generator()
# next(batch_generator)[0].shape

In [20]:
MAX_EPOCH = 300

for epoch in range(MAX_EPOCH):
    training_loss_list = list()
    batch_generator = dataset_loader_training.batch_generator()
    model.train()
    
    correct_num = 0
    total_num = 0

    for (batch, label) in batch_generator:
        optimizer.zero_grad()
        pred = model(torch.cuda.FloatTensor(batch))
        loss = F.nll_loss(pred, torch.cuda.LongTensor(label))
        loss.backward()
        training_loss_list.append(loss.item())
        optimizer.step()
        
        _, pred_label = torch.max(pred, 1)
        correct_num += int((pred_label == torch.cuda.LongTensor(label)).sum().cpu().numpy())
        total_num += len(label)
        
    acc_train = correct_num / total_num

    testing_loss_list = list()

    batch_generator = dataset_loader_testing.batch_generator()

    model.eval()
    
    correct_num = 0
    total_num = 0

    for (batch, label) in batch_generator:
    #     optimizer.zero_grad()
        pred = model(torch.cuda.FloatTensor(batch))
        loss = F.nll_loss(pred, torch.cuda.LongTensor(label))
#         loss.backward()
        testing_loss_list.append(loss.item())
    #     optimizer.step()
    
        _, pred_label = torch.max(pred, 1)
        correct_num += int((pred_label == torch.cuda.LongTensor(label)).sum().cpu().numpy())
        total_num += len(label)
        
    acc_test = correct_num / total_num

    validation_loss_list = list()

    batch_generator = dataset_loader_validation.batch_generator()
    
    correct_num = 0
    total_num = 0

    for (batch, label) in batch_generator:
    #     optimizer.zero_grad()
        pred = model(torch.cuda.FloatTensor(batch))
        loss = F.nll_loss(pred, torch.cuda.LongTensor(label))
#         loss.backward()
        validation_loss_list.append(loss.item())
    #     optimizer.step()
    
        _, pred_label = torch.max(pred, 1)
        correct_num += int((pred_label == torch.cuda.LongTensor(label)).sum().cpu().numpy())
        total_num += len(label)
        
    acc_valid = correct_num / total_num
    
    training_loss_mean = np.mean(training_loss_list)
    testing_loss_mean = np.mean(testing_loss_list)
    validation_loss_mean = np.mean(validation_loss_list)
    
    print(datetime.datetime.now().time())
    print('#{:03d} [Train: {:0.3f}] [Test: {:0.3f}] [Valid: {:0.3f}]'.format(epoch, training_loss_mean, testing_loss_mean, validation_loss_mean))
    print('#{:03d} [Train: {:0.3f}] [Test: {:0.3f}] [Valid: {:0.3f}]'.format(epoch, acc_train, acc_test, acc_valid))
    

15:56:30.275504
#000 [Train: 3.318] [Test: 3.092] [Valid: 3.059]
#000 [Train: 0.071] [Test: 0.114] [Valid: 0.126]
15:57:13.537327
#001 [Train: 2.942] [Test: 2.827] [Valid: 2.780]
#001 [Train: 0.142] [Test: 0.169] [Valid: 0.188]
15:57:56.881831
#002 [Train: 2.679] [Test: 2.579] [Valid: 2.515]
#002 [Train: 0.209] [Test: 0.236] [Valid: 0.257]
15:58:40.206625
#003 [Train: 2.443] [Test: 2.393] [Valid: 2.313]
#003 [Train: 0.278] [Test: 0.291] [Valid: 0.314]
15:59:23.417626
#004 [Train: 2.252] [Test: 2.203] [Valid: 2.143]
#004 [Train: 0.338] [Test: 0.358] [Valid: 0.372]
16:00:06.492821
#005 [Train: 2.054] [Test: 2.002] [Valid: 1.951]
#005 [Train: 0.402] [Test: 0.416] [Valid: 0.433]
16:00:49.590612
#006 [Train: 1.849] [Test: 1.813] [Valid: 1.767]
#006 [Train: 0.467] [Test: 0.477] [Valid: 0.487]
16:01:32.724622
#007 [Train: 1.668] [Test: 1.652] [Valid: 1.620]
#007 [Train: 0.516] [Test: 0.525] [Valid: 0.535]
16:02:15.978592
#008 [Train: 1.537] [Test: 1.549] [Valid: 1.525]
#008 [Train: 0.552] [Te

16:48:19.552156
#072 [Train: 0.275] [Test: 0.426] [Valid: 0.432]
#072 [Train: 0.918] [Test: 0.875] [Valid: 0.871]
16:49:02.747744
#073 [Train: 0.273] [Test: 0.431] [Valid: 0.439]
#073 [Train: 0.919] [Test: 0.871] [Valid: 0.868]
16:49:45.980086
#074 [Train: 0.267] [Test: 0.428] [Valid: 0.434]
#074 [Train: 0.921] [Test: 0.875] [Valid: 0.875]
16:50:29.088380
#075 [Train: 0.266] [Test: 0.447] [Valid: 0.431]
#075 [Train: 0.921] [Test: 0.870] [Valid: 0.878]
16:51:12.302870
#076 [Train: 0.264] [Test: 0.417] [Valid: 0.427]
#076 [Train: 0.922] [Test: 0.877] [Valid: 0.876]
16:51:55.598981
#077 [Train: 0.255] [Test: 0.412] [Valid: 0.430]
#077 [Train: 0.924] [Test: 0.883] [Valid: 0.874]
16:52:38.826036
#078 [Train: 0.258] [Test: 0.432] [Valid: 0.424]
#078 [Train: 0.923] [Test: 0.875] [Valid: 0.881]
16:53:22.064737
#079 [Train: 0.250] [Test: 0.413] [Valid: 0.422]
#079 [Train: 0.926] [Test: 0.878] [Valid: 0.878]
16:54:05.245023
#080 [Train: 0.246] [Test: 0.417] [Valid: 0.413]
#080 [Train: 0.927] [Te

17:40:12.272651
#144 [Train: 0.116] [Test: 0.400] [Valid: 0.398]
#144 [Train: 0.966] [Test: 0.892] [Valid: 0.896]
17:40:55.413494
#145 [Train: 0.114] [Test: 0.389] [Valid: 0.394]
#145 [Train: 0.968] [Test: 0.897] [Valid: 0.898]
17:41:38.547401
#146 [Train: 0.113] [Test: 0.391] [Valid: 0.402]
#146 [Train: 0.967] [Test: 0.897] [Valid: 0.893]
17:42:21.662347
#147 [Train: 0.112] [Test: 0.405] [Valid: 0.400]
#147 [Train: 0.968] [Test: 0.895] [Valid: 0.897]
17:43:04.739420
#148 [Train: 0.110] [Test: 0.391] [Valid: 0.396]
#148 [Train: 0.969] [Test: 0.897] [Valid: 0.895]
17:43:47.873393
#149 [Train: 0.109] [Test: 0.387] [Valid: 0.399]
#149 [Train: 0.969] [Test: 0.900] [Valid: 0.895]
17:44:31.005430
#150 [Train: 0.108] [Test: 0.390] [Valid: 0.393]
#150 [Train: 0.970] [Test: 0.898] [Valid: 0.897]
17:45:14.225014
#151 [Train: 0.109] [Test: 0.401] [Valid: 0.399]
#151 [Train: 0.968] [Test: 0.897] [Valid: 0.897]
17:45:57.405153
#152 [Train: 0.106] [Test: 0.402] [Valid: 0.406]
#152 [Train: 0.970] [Te

18:32:03.662980
#216 [Train: 0.052] [Test: 0.476] [Valid: 0.474]
#216 [Train: 0.986] [Test: 0.899] [Valid: 0.897]
18:32:46.824821
#217 [Train: 0.052] [Test: 0.473] [Valid: 0.481]
#217 [Train: 0.986] [Test: 0.900] [Valid: 0.896]
18:33:29.923289
#218 [Train: 0.052] [Test: 0.488] [Valid: 0.477]
#218 [Train: 0.987] [Test: 0.898] [Valid: 0.896]
18:34:13.018456
#219 [Train: 0.052] [Test: 0.483] [Valid: 0.476]
#219 [Train: 0.986] [Test: 0.898] [Valid: 0.898]
18:34:56.096037
#220 [Train: 0.051] [Test: 0.480] [Valid: 0.469]
#220 [Train: 0.987] [Test: 0.897] [Valid: 0.899]
18:35:39.191397
#221 [Train: 0.050] [Test: 0.472] [Valid: 0.477]
#221 [Train: 0.987] [Test: 0.898] [Valid: 0.900]
18:36:22.360922
#222 [Train: 0.048] [Test: 0.481] [Valid: 0.485]
#222 [Train: 0.987] [Test: 0.899] [Valid: 0.895]
18:37:05.614471
#223 [Train: 0.051] [Test: 0.460] [Valid: 0.467]
#223 [Train: 0.986] [Test: 0.901] [Valid: 0.897]
18:37:48.852206
#224 [Train: 0.049] [Test: 0.477] [Valid: 0.473]
#224 [Train: 0.986] [Te

19:23:52.496493
#288 [Train: 0.021] [Test: 0.581] [Valid: 0.574]
#288 [Train: 0.994] [Test: 0.897] [Valid: 0.895]
19:24:35.637058
#289 [Train: 0.023] [Test: 0.584] [Valid: 0.594]
#289 [Train: 0.994] [Test: 0.898] [Valid: 0.895]
19:25:18.699941
#290 [Train: 0.023] [Test: 0.571] [Valid: 0.566]
#290 [Train: 0.994] [Test: 0.898] [Valid: 0.895]
19:26:01.803902
#291 [Train: 0.021] [Test: 0.569] [Valid: 0.576]
#291 [Train: 0.995] [Test: 0.901] [Valid: 0.893]
19:26:44.879987
#292 [Train: 0.021] [Test: 0.584] [Valid: 0.569]
#292 [Train: 0.995] [Test: 0.894] [Valid: 0.894]
19:27:28.058027
#293 [Train: 0.020] [Test: 0.562] [Valid: 0.566]
#293 [Train: 0.995] [Test: 0.897] [Valid: 0.898]
19:28:11.190942
#294 [Train: 0.021] [Test: 0.575] [Valid: 0.572]
#294 [Train: 0.995] [Test: 0.899] [Valid: 0.897]
19:28:54.313566
#295 [Train: 0.019] [Test: 0.569] [Valid: 0.579]
#295 [Train: 0.995] [Test: 0.898] [Valid: 0.897]
19:29:37.392798
#296 [Train: 0.019] [Test: 0.586] [Valid: 0.579]
#296 [Train: 0.995] [Te

In [21]:
# label.shape

In [22]:
# output = model(torch.cuda.FloatTensor(batch))

In [23]:
# output.shape

In [24]:
# torch.tensor(batch).unsqueeze_(1).shape

In [25]:
# input_tensor = torch.tensor(batch).to(device).float()
# input_tensor = input_tensor.transpose(1, 2)
# model(input_tensor).shape

In [26]:
# for batch, label_list in batch_generator:
#     print(batch.shape[0])
#     plt.figure(figsize=(5, 1))
#     plt.plot(label_list)
#     plt.show()
    
#     plt.figure(figsize=(6, 18))
# #     plt.imshow(batch[0], aspect='auto')
#     plt.imshow(batch[0])
#     plt.show()
    
#     break